In [1]:
import torch

class ModelExporter:
    """
    A utility class to export a PyTorch model to an ONNX file.
    """
    def __init__(self, model_name):
        """
        Initialize the exporter with a model name.
        Args:
            model_name (str): The name of the model. This will be used as the ONNX file name.
        """
        self.model_name = model_name

    def export_to_onnx(self, model, dummy_input):
        """
        Exports the given PyTorch model to an ONNX file.
        Args:
            model (torch.nn.Module): The PyTorch model to be exported.
            dummy_input (torch.Tensor): A dummy input tensor that matches the input shape of the model.
        """
        # Generate the ONNX file name using the model name
        onnx_file_name = f"model_onnx/{self.model_name}.onnx"

        # Export the model to ONNX format
        torch.onnx.export(
            model,
            dummy_input,
            onnx_file_name,           # Name of the output ONNX file
            export_params=True,       # Include the trained parameters in the exported file
            opset_version=11,         # Specify the ONNX opset version
            do_constant_folding=True, # Perform constant folding optimization
            input_names=["input"],    # Name of the input tensor
            output_names=["output"]   # Name of the output tensor
        )
        print(f"The ONNX file has been saved as '{onnx_file_name}'.")

In [2]:
import torch

class CustomReLU(torch.nn.Module):
    """
    Custom implementation of the ReLU activation function.
    """
    def __init__(self):
        super(CustomReLU, self).__init__()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Applies the ReLU function element-wise: max(0, x).
        Args:
            x (torch.Tensor): Input tensor
        Returns:
            torch.Tensor: Output tensor with ReLU applied
        """
        return torch.maximum(x, torch.zeros_like(x))

In [3]:
import torch

class ConvolutionalLayer:
    """
    A utility class to create a convolutional layer with a custom ReLU activation.
    """
    @staticmethod
    def create(in_channels: int, out_channels: int, kernel_size: int, stride: int, padding: int) -> torch.nn.Sequential:
        """
        Creates a convolutional layer with a custom ReLU activation.
        Args:
            in_channels (int): Number of input channels
            out_channels (int): Number of output channels
            kernel_size (int): Size of the convolution kernel
            stride (int): Stride of the convolution
            padding (int): Padding added to the input
        Returns:
            torch.nn.Sequential: A sequential layer containing Conv2d and CustomReLU(+Batch Normalization)
        """
        return torch.nn.Sequential(
            torch.nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding),
            torch.nn.BatchNorm2d(out_channels),
            CustomReLU()
        )

In [4]:
import torch

class FullyConnectedLayer:
    """
    A utility class to create Fully Connected (FC) layers with Xavier initialization.
    """
    @staticmethod
    def Dense(input_dim: int, output_dim: int) -> torch.nn.Linear:
        """
        Creates an FC layer and applies Xavier initialization.
        Args:
            input_dim (int): Number of input features
            output_dim (int): Number of output features
        Returns:
            torch.nn.Linear: Initialized Fully-Connected layer
        """
        layer = torch.nn.Linear(input_dim, output_dim, bias=True)
        torch.nn.init.xavier_uniform_(layer.weight)
        return layer

In [5]:
import torch

class CustomZFNet(torch.nn.Module):
    """
    Implementation of the introduced by ZFNet model.
    Input: Image tensor (batch_size, 3, 224, 224)
    Output: Class scores (batch_size, 1000)
    """
    def __init__(self):
        super(CustomZFNet, self).__init__()

        # Convolutional and pooling layers
        self.layer1 = torch.nn.Sequential(
            ConvolutionalLayer.create(3, 96, 7, 2, 1),
            torch.nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )

        self.layer2 = torch.nn.Sequential(
            ConvolutionalLayer.create(96, 256, 5, 2, 0),
            torch.nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )

        self.layer3 = torch.nn.Sequential(
            ConvolutionalLayer.create(256, 384, 3, 1, 1),
            ConvolutionalLayer.create(384, 384, 3, 1, 1),
            ConvolutionalLayer.create(384, 256, 3, 1, 1),
            torch.nn.MaxPool2d(kernel_size=3, stride=2, padding=0)
        )

        # Fully Connected layer
        self.layer4 = FullyConnectedLayer.Dense(6 * 6 * 256, 4096)

        self.layer5 = FullyConnectedLayer.Dense(4096, 4096)

        self.layer6 = FullyConnectedLayer.Dense(4096, 1000)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Defines the forward pass of the model.
        Args:
            x (torch.Tensor): Input image tensor (batch_size, 3, 224, 224)
        Returns:
            torch.Tensor: Class scores (batch_size, 1000)
        """
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.layer6(x)
        return x

In [6]:
from torchsummary import summary

model = CustomZFNet()

print(model.__class__.__name__)
summary(model, input_size=(3, 224, 224))

CustomZFNet
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 96, 110, 110]          14,208
       BatchNorm2d-2         [-1, 96, 110, 110]             192
        CustomReLU-3         [-1, 96, 110, 110]               0
         MaxPool2d-4           [-1, 96, 55, 55]               0
            Conv2d-5          [-1, 256, 26, 26]         614,656
       BatchNorm2d-6          [-1, 256, 26, 26]             512
        CustomReLU-7          [-1, 256, 26, 26]               0
         MaxPool2d-8          [-1, 256, 13, 13]               0
            Conv2d-9          [-1, 384, 13, 13]         885,120
      BatchNorm2d-10          [-1, 384, 13, 13]             768
       CustomReLU-11          [-1, 384, 13, 13]               0
           Conv2d-12          [-1, 384, 13, 13]       1,327,488
      BatchNorm2d-13          [-1, 384, 13, 13]             768
       CustomReLU-14       